In [71]:
from tqdm.auto import tqdm
import time
import subprocess
import random

In [72]:
wa = False

class Test():
    def __init__(self):
        self.generate_test()
        
    def generate_test(self): 
        with open("test.in", 'w') as test:
            self.n = random.randint(3, 4)
            self.p = random.randint(1, 2)
            self.mod = random.randint(100, 1_000_000)
            
            test.write(f"{self.n} {self.p} {self.mod}\n")
            
            for _ in range(self.p):
                for _ in range(3):
                    row = "".join('x' if random.random() < 0.5 else '.' for _ in range(3))
                    test.write(row + "\n")
        
    def check_solutions(self, a_out, b_out):
        global wa
        # a: brut, b: better_solution
        with open(a_out, 'r') as a_out, open(b_out, 'r') as b_out:
            self.test_data_brut = int(a_out.readline().strip())
            self.test_data_chad = int(b_out.readline().strip())
                
            self.better_check()
    
    def better_check(self):
        global wa
        if self.test_data_brut != self.test_data_chad:
            print("WA")
            wa = True
            return

In [73]:
# convention a: brut, b: better_solutoin
class Referee():
    def __init__(self, program_a: str, program_b: str, max_time):
        # program a = sth.cpp (give full name with .cpp)
        
        self.program_a = program_a
        self.program_b = program_b
        self.max_time = max_time
        
        # compile programs 
        self.exe_a = self.compile(program_a)
        self.exe_b = self.compile(program_b)
        
    def compile(self, program: str) -> str:
        exe_name = program.split('.')[0]
        compile_command = ['g++', program, '-o', exe_name]
        subprocess.run(compile_command, check=True)
        return exe_name
    
    def do_tests(self): 
        self.times = []
        for _ in tqdm(range(2000)): 
            # create test 
            self.test = Test()
            
            # test program_a
            start_time_a = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_a}.out", "w") as output_a:
                try:
                    subprocess.run(f"./{self.exe_a}", stdin=test_input, stdout=output_a, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_a} exceeded time limit")
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_a} ended with an error")
                    exit(2)
            end_time_a = time.time()
            
            # test program_b
            start_time_b = time.time()
            with open("test.in", "r") as test_input, open(f"{self.exe_b}.out", "w") as output_b:
                try:
                    subprocess.run(f"./{self.exe_b}", stdin=test_input, stdout=output_b, stderr=subprocess.DEVNULL, timeout=self.max_time * 2, check=True)
                except subprocess.TimeoutExpired:
                    print(f"{self.exe_b} exceeded time limit") 
                    exit(1)
                except subprocess.CalledProcessError:
                    print(f"{self.exe_b} ended with an error")
                    exit(2)
            end_time_b = time.time()
            
            
            # check solution
            global wa
            wa = False
            self.test.check_solutions(f"{self.exe_a}.out", f"{self.exe_b}.out")
            if wa == True:
               break 
            
            # solutions are ok
            self.times.append([end_time_a - start_time_a, end_time_b - start_time_b])
        
        else:
            print("everything ok 🚀") 

In [74]:
referee = Referee('brut_solution.cpp', 'gigachad_solution.cpp', 2)
referee.do_tests()

100%|██████████| 2000/2000 [03:32<00:00,  9.40it/s]

everything ok 🚀


In [75]:
import pandas as pd
df = pd.DataFrame(referee.times)
df.columns = ['brut', 'chad']
df = df.sort_values(by='chad', ascending=False)
df

,brut,chad
430,0.292575,0.139684
1068,0.047116,0.032523
1069,0.288800,0.030516
1467,0.127317,0.021198
7,0.181604,0.020702
...,...,...
1214,0.017249,0.006413
1318,0.018109,0.006398
702,0.016330,0.006385
1389,0.016883,0.006381


In [76]:
df.mean()
df.sort_values(by='brut', ascending=False)

,brut,chad
522,0.297375,0.012197
516,0.296523,0.013186
430,0.292575,0.139684
539,0.288860,0.017713
1069,0.288800,0.030516
...,...,...
593,-2.135943,0.008410
1440,-2.215400,0.009101
1152,-2.290623,0.013471
1718,-2.305947,0.008778
